In [ ]:
import networkx as nx
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
from torch_geometric import datasets
from torch_geometric.nn import GCNConv
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import scatter
from torch_geometric.datasets import Planetoid

In [ ]:
class CustomDs(torch_geometric.data.InMemoryDataset):
    def __init__(self, n_graphs, tranform=None) -> None:
        graphs = []
        for _ in range(n_graphs):
            super().__init__(".", tranform)
            g = nx.erdos_renyi_graph(100, 0.1)
            nx.set_node_attributes(g, {i[0]: (i[1]>9)*1 for i in list(g.degree())}, "health")
            n_nodes = len(g.nodes())
            edge_from = [i[0] for i in g.edges]
            edge_to = [i[1] for i in g.edges]

            ei = torch.tensor([
            edge_from,
            edge_to
            ], dtype=torch.long)

            y = np.array(list(dict(g.nodes(data="health")).values())).reshape(-1,1)
            y = torch.tensor(y).float()
            # x = torch.rand(size=(n_nodes,1))
            x = torch.tensor(list(g.degree())).float()

            data = Data(y=y, edge_index=ei, x=x)
            graphs.append(data)

        self.data, self.slices = self.collate(graphs)

In [ ]:
ds = CustomDs(5)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(ds.num_node_features, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return torch.sigmoid(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = ds[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.binary_cross_entropy(out, data.y)
    # loss = F.ll_lo(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print((((model(ds[1]) > 0.5 )== ds[1].y).sum())-ds[1].y.sum())

tensor(23.)
tensor(24.)
tensor(23.)
tensor(23.)
tensor(22.)
tensor(23.)
tensor(23.)
tensor(22.)
tensor(23.)
tensor(22.)
tensor(22.)
tensor(22.)
tensor(21.)
tensor(23.)
tensor(22.)
tensor(21.)
tensor(21.)
tensor(22.)
tensor(23.)
tensor(23.)


In [ ]:
((model(ds[1]) > 0.5 )== ds[1].y).sum()

tensor(74)

In [ ]:
ds[1].y.sum()

tensor(51.)